In [1]:
import polars as pl
import polars_distance as pld

In [2]:
pl.Config.load_from_file("../../polars_cfg.json")

In [22]:
pl.Config.set_tbl_rows(50)

polars.config.Config

In [3]:
# data file paths
clean_summary_path = "../../data/clean_summary_data.csv"
clean_summary_incomplete_path = "../../data/clean_summary_data_incomplete.csv"

In [4]:
df_summary = pl.read_csv(clean_summary_path)
df_summary_incomplete = pl.read_csv(clean_summary_incomplete_path)

df_summary = pl.concat([df_summary, df_summary_incomplete])

# training_set = pl.read_csv("../data/ARC_training_tasks.csv")
# evaluation_set = pl.read_csv("../data/ARC_evaluation_tasks.csv")

In [5]:
last_correct_joint_id = df_summary.filter(
    (pl.col("solved") == 1) & (pl.col("attempt_number") == 3)
).select("joint_id_task")
df_filtered = df_summary.filter(pl.col("joint_id_task").is_in(last_correct_joint_id))

In [6]:
df_filtered.head()

exp_name,task_type,hashed_id,joint_id_task,task_name,task_number,attempt_number,num_actions,solved,test_output_grid,first_written_solution,last_written_solution,complete
str,str,str,str,str,i64,i64,i64,bool,str,str,str,bool
"""expv0""","""training""","""2b06355cea411af7a6fa212c90eb3c6d""","""2b06355cea411af7a6fa212c90eb3c6d_228f6490.jso…","""228f6490.json""",1,1,111,false,"""|0055555002|2052225000|0055255000|0055555000|…","""Move the colored shapes into the empty black …","""Moved the colored shapes into the empty black…",true
"""expv0""","""training""","""2b06355cea411af7a6fa212c90eb3c6d""","""2b06355cea411af7a6fa212c90eb3c6d_228f6490.jso…","""228f6490.json""",1,2,117,false,"""|0055555002|2057775000|0055755000|0055555000|…","""Move the colored shapes into the empty black …","""Moved the colored shapes into the empty black…",true
"""expv0""","""training""","""2b06355cea411af7a6fa212c90eb3c6d""","""2b06355cea411af7a6fa212c90eb3c6d_228f6490.jso…","""228f6490.json""",1,3,120,true,"""|0055555002|2057775000|0055755000|0055555000|…","""Move the colored shapes into the empty black …","""Moved the colored shapes into the empty black…",true
"""expv0""","""training""","""7c3f0d5ec05185be127ab78876395426""","""7c3f0d5ec05185be127ab78876395426_e50d258f.jso…","""e50d258f.json""",4,1,23,false,"""|1282|8818|1281|""","""The bottom box gets magnified.""","""One of the 3 shapes gets magnified. It took t…",true
"""expv0""","""training""","""7c3f0d5ec05185be127ab78876395426""","""7c3f0d5ec05185be127ab78876395426_e50d258f.jso…","""e50d258f.json""",4,2,52,false,"""|2888|8818|1281|8888|""","""The bottom box gets magnified.""","""One of the 3 shapes gets magnified. It took t…",true


In [15]:
df_distances = (
    df_filtered.group_by("joint_id_task")
    .agg(
        [
            # Get all test outputs ordered by attempt_number
            pl.col("test_output_grid")
            .sort_by("attempt_number")
            .alias("outputs")
        ]
    )
    .with_columns(
        [
            # Get the first, second, and third outputs
            pl.col("outputs").list.get(0).alias("output1"),
            pl.col("outputs").list.get(1).alias("output2"),
            pl.col("outputs").list.get(2).alias("output3"),
        ]
    )
    .with_columns(
        [
            # Calculate distances between consecutive attempts
            pl.col("output1")
            .dist_str.levenshtein("output2", normalized=True)
            .alias("dist_1_to_2"),
            pl.col("output2")
            .dist_str.levenshtein("output3", normalized=True)
            .alias("dist_2_to_3"),
        ]
    )
    .drop("outputs")
)

df_distances = df_distances.with_columns(
    pl.mean_horizontal("dist_1_to_2", "dist_2_to_3").alias("mean_dist")
)

In [24]:
df_distances.sort("mean_dist", descending=True).filter(
    (pl.col("mean_dist") < 0.5) & (pl.col("dist_2_to_3") < pl.col("dist_1_to_2"))
).slice(20, 40)

joint_id_task,output1,output2,output3,dist_1_to_2,dist_2_to_3,mean_dist
str,str,str,str,f64,f64,f64
"""0e42f7edd4f8206b4f09a049bae44bc6_d4b1c2b1.jso…","""|777111777|777111777|777111777|333333666|3333…","""|777711117777|777711117777|777711117777|77771…","""|777771111177777|777771111177777|777771111177…",0.420382,0.348548,0.384465
"""da0da70f01db728bb465ab904a135d9e_1c786137.jso…","""|0000000000000000|0000000000000000|0000000000…","""|208111010|810828128|820201181|018811018|0188…","""|208111010|810828128|820201181|018811818|0188…",0.731034,0.037037,0.384036
"""2025eb689cc6a1f4db636ca6f6710fc3_d13f3404.jso…","""|903|490|049|""","""|903000|490300|049030|004903|000490|000049|""","""|003000|000300|049030|004903|000490|000049|""",0.697674,0.069767,0.383721
"""d692f427dcd1a02b5b4faa2fd698291b_09629e4f.jso…","""|66653335000|66653335000|66653335000|55555555…","""|66654445000|66654445000|66654445000|55555555…","""|00052225444|00052225444|00052225444|55555555…",0.406015,0.360902,0.383459
"""e4bae42f79445ed57408a511f96764a6_358ba94e.jso…","""|111|101|111|""","""|11111|11111|10111|11111|11111|""","""|11111|10101|11111|10101|11111|""",0.580645,0.16129,0.370968
"""0d950d9ed094a253ab4d72c033871197_62b74c02.jso…","""|38833333333333|21122222222222|13311111111111…","""|38833333333883|21123333332112|13313333331331…","""|38833333333883|21122222222112|13311111111331…",0.42623,0.295082,0.360656
"""f076bb4cb6e46d77872910f2120690e4_a79310a0.jso…","""|000|000|000|""","""|00000|00800|08800|00800|00000|""","""|00000|00200|02200|00200|00000|""",0.580645,0.129032,0.354839
"""6026d2563d049de8566b85ce9dd27830_d0f5fe59.jso…","""|800|080|008|""","""|8000|0800|0080|0008|""","""|80000|08000|00800|00080|00008|""",0.380952,0.322581,0.351767
"""ff1070a18395ae11bf95c9aff9290279_195ba7dc.jso…","""|777007|770707|777707|777700|007777|""","""|111101|110001|111101|111100|011111|""","""|111001|110101|111101|111100|011111|""",0.638889,0.055556,0.347222


1: e6721834.json
2: d931c21c.json 3. 3af2c5a8.json


In [27]:
df_summary.filter(pl.col("last_written_solution").str.contains("wavy"))

exp_name,task_type,hashed_id,joint_id_task,task_name,task_number,attempt_number,num_actions,solved,test_output_grid,first_written_solution,last_written_solution,complete
str,str,str,str,str,i64,i64,i64,bool,str,str,str,bool
"""expv6_1""","""evaluation""","""23d102b56921182b4221066a6b64684a""","""23d102b56921182b4221066a6b64684a_1c56ad9f.jso…","""1c56ad9f.json""",3,1,48,false,"""|00000000000000|00007777777000|00070070070000…","""Keep the top and bottom lines the same. the l…","""Keep the upper and lower lines the same. Move…",true
"""expv6_1""","""evaluation""","""23d102b56921182b4221066a6b64684a""","""23d102b56921182b4221066a6b64684a_1c56ad9f.jso…","""1c56ad9f.json""",3,2,122,false,"""|00000000000000|00007777777000|00000700700700…","""Keep the top and bottom lines the same. the l…","""Keep the upper and lower lines the same. Move…",true
"""expv6_1""","""evaluation""","""23d102b56921182b4221066a6b64684a""","""23d102b56921182b4221066a6b64684a_1c56ad9f.jso…","""1c56ad9f.json""",3,3,124,false,"""|00000000000000|00007777777000|00000700700700…","""Keep the top and bottom lines the same. the l…","""Keep the upper and lower lines the same. Move…",true
